## Preprocessing

In [17]:
# OPTMISING CHANGES
# ATTEMPT 1:
# Increased the cutoff values: Changed the cutoff values for APPLICATION_TYPE and CLASSIFICATION to group more rare occurrences into "Other". This simplifies the model and reduces noise.
# Added more neurons and hidden layers: Increased the complexity of the neural network by adding a third hidden layer and increasing the number of neurons in each layer.
# Increased the number of epochs: Trained the model for more epochs (150) to allow it to learn more from the data.

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [75]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_dropped = application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS'])

In [76]:
# Determine the number of unique values in each column.
unique_values = application_df_dropped.nunique()
unique_values

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [77]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df_dropped['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [78]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# changed Bin cutoff value (other) from 200 to 500.
application_types_to_replace = application_type_counts[application_type_counts < 600].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_dropped['APPLICATION_TYPE'] = application_df_dropped['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df_dropped['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [79]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df_dropped['CLASSIFICATION'].value_counts()

classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [80]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [81]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Changed Bin cutoff value (other) from 900 to 1000.
classifications_to_replace = classification_counts[classification_counts < 1000].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df_dropped['CLASSIFICATION'] = application_df_dropped['CLASSIFICATION'].replace(cls, "Other")


# Check to make sure replacement was successful
application_df_dropped['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [82]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df_dropped)

In [83]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(columns=['IS_SUCCESSFUL'])
y = application_df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [84]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [148]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
# Changed the number of neurons in each hidden layer. (increased number of neurons)

nn.add(tf.keras.layers.Dense(units=500, activation='elu', input_dim=X_train_scaled.shape[1]))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=500, activation='tanh'))

# Added another hidden layer.

# # Third hidden layer
nn.add(tf.keras.layers.Dense(units=500, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 500)               20500     
                                                                 
 dense_79 (Dense)            (None, 500)               250500    
                                                                 
 dense_80 (Dense)            (None, 500)               250500    
                                                                 
 dense_81 (Dense)            (None, 1)                 501       
                                                                 
Total params: 522,001
Trainable params: 522,001
Non-trainable params: 0
_________________________________________________________________


In [149]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
# Increased the number of epochs to allow the model to train more iterations over the data.
history = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 5s 4ms/step - loss: 0.5821 - accuracy: 0.7151
Epoch 2/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5613 - accuracy: 0.7285
Epoch 3/100
858/858 [==============================] - 9s 10ms/step - loss: 0.5583 - accuracy: 0.7289
Epoch 4/100
858/858 [==============================] - 5s 6ms/step - loss: 0.5567 - accuracy: 0.7270
Epoch 5/100
858/858 [==============================] - 4s 5ms/step - loss: 0.5546 - accuracy: 0.7312
Epoch 6/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5531 - accuracy: 0.7299
Epoch 7/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5511 - accuracy: 0.7314
Epoch 8/100
858/858 [==============================] - 4s 5ms/step - loss: 0.5514 - accuracy: 0.7310
Epoch 9/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5496 - accuracy: 0.7313
Epoch 10/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5489 - accur

In [147]:
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model(activation1='relu', activation2='relu', activation3='relu'):
    nn = tf.keras.models.Sequential()
    nn.add(tf.keras.layers.Dense(units=500, activation=activation1, input_dim=X_train_scaled.shape[1]))
    nn.add(tf.keras.layers.Dense(units=500, activation=activation2))
    nn.add(tf.keras.layers.Dense(units=500, activation=activation3))
    nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return nn

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

param_distributions = {
    'activation1': ['relu', 'tanh', 'elu'],
    'activation2': ['relu', 'tanh', 'elu'],
    'activation3': ['relu', 'tanh', 'elu']
}

random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=9, n_jobs=-1, cv=3, random_state=42)
random_search_result = random_search.fit(X_train_scaled, y_train)

print(f"Best Score: {random_search_result.best_score_}")
print(f"Best Params: {random_search_result.best_params_}")

C:\Users\morga\AppData\Local\Temp\ipykernel_19440\1601978430.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)


Best Score: 0.7308939695358276
Best Params: {'activation3': 'relu', 'activation2': 'tanh', 'activation1': 'elu'}


In [146]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5662 - accuracy: 0.7230 - 634ms/epoch - 3ms/step
Loss: 0.5662013292312622, Accuracy: 0.7230320572853088


In [38]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")